- how to treat # and other punctuation
- Implement BPE
- Implement WordPiece
- Implement Spacy
- Implement Unigram
- Compare size of vocab
- Compare performance of model

In [1]:
import pandas as pd

In [34]:
df_comments_bpe = pd.read_csv("/Users/fabianmahner/dlss-project24/data/preprocessed/comments.csv")
df_comments_wp = pd.read_csv("/Users/fabianmahner/dlss-project24/data/preprocessed/comments.csv")
df_comments_spacy = pd.read_csv("/Users/fabianmahner/dlss-project24/data/preprocessed/comments.csv")
df_comments_unigram = pd.read_csv("/Users/fabianmahner/dlss-project24/data/preprocessed/comments.csv")
df_comments = pd.read_csv("/Users/fabianmahner/dlss-project24/data/preprocessed/comments.csv")

In [4]:
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, trainers, processors


# For comments

In [5]:
# Initialize a tokenizer
tokenizer = Tokenizer(models.BPE())

# Use a pre-tokenizer to split text into words
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

# Define a trainer
trainer = trainers.BpeTrainer(vocab_size=10000, min_frequency=2, special_tokens=["<unk>", "<s>", "</s>", "<pad>", "<mask>"])

# Train the tokenizer on the dataframe's text column
texts = df_comments_bpe["body"].tolist()
tokenizer.train_from_iterator(texts, trainer)

In [6]:
# Function to tokenize a single text
def tokenize_text(text):
    return tokenizer.encode(text).tokens

# Apply the function to the dataframe
df_comments_bpe['tokens'] = df_comments_bpe['body'].apply(tokenize_text)

# Display the dataframe with tokens
df_comments_bpe

,Unnamed: 0,id,body,cleaned,lemmatized,tokens
0,0,f1bk4k3,https://www.penmediainc.com/2019/09/23/climate...,NaN,NaN,"[https, ://, www, ., pen, medi, ain, c, ., com..."
1,1,frqfnw7,"Nice try, but false equivalency is still false...",nice try but false equivalency is still false ...,nice try but false equivalency be still false ...,"[Nice, try, ,, but, false, equivalency, is, st..."
2,2,dh9dp98,"I'm slightly torn on this, the idea of nations...",i slightly torn on this the idea of nations tr...,I slightly tear on this the idea of nation try...,"[I, ', m, slightly, torn, on, this, ,, the, id..."
3,3,i1vluk4,*Article contents as always:*\n\n&amp;#x200B;\...,article contents as always south australian pr...,article content as always south australian pre...,"[*, Artic, le, cont, ents, as, always, :, *, &..."
4,4,fr9nzyu,What is the current outlook for ocean acidific...,what is the current outlook for ocean acidific...,what be the current outlook for ocean acidific...,"[What, is, the, current, out, look, for, ocean..."
...,...,...,...,...,...,...
1995,1995,h8859gb,The usual reason - their policies most closely...,the usual reason their policies most closely a...,the usual reason their policy most closely ali...,"[The, usual, reason, -, their, policies, most,..."
1996,1996,fqo3dlb,"it's not deep though, it is a surface level ob...",it not deep though it is a surface level obser...,it not deep though it be a surface level obser...,"[it, ', s, not, deep, though, ,, it, is, a, su..."
1997,1997,fhhct09,"**Hello u/UltimateWebRedditor, unfortunately y...",hello unfortunately your submission do blame c...,hello unfortunately your submission do blame c...,"[**, Hello, u, /, Ultim, ate, We, b, Red, di, ..."
1998,1998,h4s5us0,&gt; New conventional nuclear plants... exorbi...,new conventional nuclear plants exorbitant cos...,new conventional nuclear plant exorbitant cost...,"[&, gt, ;, New, conventional, nuclear, plants,..."


## Word Piece

In [7]:
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, trainers, normalizers

# Initialize a tokenizer
tokenizer = Tokenizer(models.WordPiece(unl_token="[UNK]"))

# Normalize the text (optional but recommended)
tokenizer.normalizer = normalizers.BertNormalizer()

# Use a pre-tokenizer to split text into words
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

# Define a trainer
trainer = trainers.WordPieceTrainer(vocab_size=10000, min_frequency=2, special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])

# Train the tokenizer on the dataframe's text column
texts = df_comments_wp['body'].tolist()
tokenizer.train_from_iterator(texts, trainer)

Ignored unknown kwargs option unl_token





In [8]:
# Function to tokenize a single text
def tokenize_text(text):
    return tokenizer.encode(text).tokens

# Apply the function to the dataframe
df_comments_wp['tokens'] = df_comments_wp['body'].apply(tokenize_text)

# Display the dataframe with tokens
df_comments_wp

,Unnamed: 0,id,body,cleaned,lemmatized,tokens
0,0,f1bk4k3,https://www.penmediainc.com/2019/09/23/climate...,NaN,NaN,"[https, ://, www, ., pen, ##medi, ##ain, ##c, ..."
1,1,frqfnw7,"Nice try, but false equivalency is still false...",nice try but false equivalency is still false ...,nice try but false equivalency be still false ...,"[nice, try, ,, but, false, equivalency, is, st..."
2,2,dh9dp98,"I'm slightly torn on this, the idea of nations...",i slightly torn on this the idea of nations tr...,I slightly tear on this the idea of nation try...,"[i, ', m, slightly, torn, on, this, ,, the, id..."
3,3,i1vluk4,*Article contents as always:*\n\n&amp;#x200B;\...,article contents as always south australian pr...,article content as always south australian pre...,"[*, article, content, ##s, as, always, :, ##*,..."
4,4,fr9nzyu,What is the current outlook for ocean acidific...,what is the current outlook for ocean acidific...,what be the current outlook for ocean acidific...,"[what, is, the, current, outl, ##oo, ##k, for,..."
...,...,...,...,...,...,...
1995,1995,h8859gb,The usual reason - their policies most closely...,the usual reason their policies most closely a...,the usual reason their policy most closely ali...,"[the, usual, reason, -, their, policies, most,..."
1996,1996,fqo3dlb,"it's not deep though, it is a surface level ob...",it not deep though it is a surface level obser...,it not deep though it be a surface level obser...,"[it, ', s, not, deep, though, ,, it, is, a, su..."
1997,1997,fhhct09,"**Hello u/UltimateWebRedditor, unfortunately y...",hello unfortunately your submission do blame c...,hello unfortunately your submission do blame c...,"[**, hello, u, /, ultimate, ##web, ##reddit, #..."
1998,1998,h4s5us0,&gt; New conventional nuclear plants... exorbi...,new conventional nuclear plants exorbitant cos...,new conventional nuclear plant exorbitant cost...,"[&, gt, ;, new, conventional, nuclear, plants,..."


## Spacy

In [15]:
from transformers import BertTokenizer


In [20]:
# Load the pre-trained BERT tokenizer
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/Users/fabianmahner/dlss-project24/venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [21]:
import spacy
from spacy.tokens import Doc

# Define a custom tokenizer function
def custom_tokenizer(nlp):
    def tokenizer(text):
        tokens = bert_tokenizer.tokenize(text)
        return Doc(nlp.vocab, words=tokens)
    return tokenizer

# Create a spaCy pipeline and replace its tokenizer with the custom tokenizer
nlp = spacy.blank("en")
nlp.tokenizer = custom_tokenizer(nlp)


In [22]:
# Function to tokenize a single text
def tokenize_text(text):
    doc = nlp(text)
    return [token.text for token in doc]

# Apply the function to the dataframe
df_comments_spacy['tokens'] = df_comments_spacy['body'].apply(tokenize_text)

In [23]:
df_comments_spacy

,Unnamed: 0,id,body,cleaned,lemmatized,tokens
0,0,f1bk4k3,https://www.penmediainc.com/2019/09/23/climate...,NaN,NaN,"[https, :, /, /, www, ., pen, ##media, ##in, #..."
1,1,frqfnw7,"Nice try, but false equivalency is still false...",nice try but false equivalency is still false ...,nice try but false equivalency be still false ...,"[nice, try, ,, but, false, e, ##qui, ##vale, #..."
2,2,dh9dp98,"I'm slightly torn on this, the idea of nations...",i slightly torn on this the idea of nations tr...,I slightly tear on this the idea of nation try...,"[i, ', m, slightly, torn, on, this, ,, the, id..."
3,3,i1vluk4,*Article contents as always:*\n\n&amp;#x200B;\...,article contents as always south australian pr...,article content as always south australian pre...,"[*, article, contents, as, always, :, *, &, am..."
4,4,fr9nzyu,What is the current outlook for ocean acidific...,what is the current outlook for ocean acidific...,what be the current outlook for ocean acidific...,"[what, is, the, current, outlook, for, ocean, ..."
...,...,...,...,...,...,...
1995,1995,h8859gb,The usual reason - their policies most closely...,the usual reason their policies most closely a...,the usual reason their policy most closely ali...,"[the, usual, reason, -, their, policies, most,..."
1996,1996,fqo3dlb,"it's not deep though, it is a surface level ob...",it not deep though it is a surface level obser...,it not deep though it be a surface level obser...,"[it, ', s, not, deep, though, ,, it, is, a, su..."
1997,1997,fhhct09,"**Hello u/UltimateWebRedditor, unfortunately y...",hello unfortunately your submission do blame c...,hello unfortunately your submission do blame c...,"[*, *, hello, u, /, ultimate, ##we, ##bre, ##d..."
1998,1998,h4s5us0,&gt; New conventional nuclear plants... exorbi...,new conventional nuclear plants exorbitant cos...,new conventional nuclear plant exorbitant cost...,"[&, gt, ;, new, conventional, nuclear, plants,..."


## Unigram

In [29]:
import sentencepiece as spm

# Save text data to a file as required by sentencepiece
with open('text_data.txt', 'w') as f:
    for text in df_comments_unigram['body']:
        f.write(text + '\n')

# Train a Unigram tokenizer
spm.SentencePieceTrainer.Train('--input=text_data.txt --model_prefix=unigram --vocab_size=2000 --model_type=unigram')


sentencepiece_trainer.cc(178) LOG(INFO) Running command: --input=text_data.txt --model_prefix=unigram --vocab_size=2000 --model_type=unigram
sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: text_data.txt
  input_format: 
  model_prefix: unigram
  model_type: UNIGRAM
  vocab_size: 2000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pa

In [30]:
from spacy.tokens import Doc

# Load the trained Unigram model
sp = spm.SentencePieceProcessor()
sp.Load("unigram.model")

# Define a custom tokenizer function
def custom_tokenizer(nlp):
    def tokenizer_func(text):
        tokens = sp.encode_as_pieces(text)
        return Doc(nlp.vocab, words=tokens)
    return tokenizer_func

# Create a spaCy pipeline and replace its tokenizer with the custom tokenizer
nlp = spacy.blank("en")
nlp.tokenizer = custom_tokenizer(nlp)


In [33]:
def tokenize_text(text):
    doc = nlp(text)
    return [token.text for token in doc]

# Apply the function to the dataframe
df_comments_unigram['tokens'] = df_comments_unigram['body'].apply(tokenize_text)

# Display the dataframe with tokens
df_comments_unigram


,Unnamed: 0,id,body,cleaned,lemmatized,tokens
0,0,f1bk4k3,https://www.penmediainc.com/2019/09/23/climate...,NaN,NaN,"[▁http, s, ://, www, ., p, en, me, di, ain, c,..."
1,1,frqfnw7,"Nice try, but false equivalency is still false...",nice try but false equivalency is still false ...,nice try but false equivalency be still false ...,"[▁N, ice, ▁try, ,, ▁but, ▁false, ▁e, q, u, i, ..."
2,2,dh9dp98,"I'm slightly torn on this, the idea of nations...",i slightly torn on this the idea of nations tr...,I slightly tear on this the idea of nation try...,"[▁I, ', m, ▁, s, l, ight, ly, ▁to, r, n, ▁on, ..."
3,3,i1vluk4,*Article contents as always:*\n\n&amp;#x200B;\...,article contents as always south australian pr...,article content as always south australian pre...,"[▁*, A, r, tic, le, ▁, content, s, ▁as, ▁alway..."
4,4,fr9nzyu,What is the current outlook for ocean acidific...,what is the current outlook for ocean acidific...,what be the current outlook for ocean acidific...,"[▁What, ▁is, ▁the, ▁current, ▁out, l, ook, ▁fo..."
...,...,...,...,...,...,...
1995,1995,h8859gb,The usual reason - their policies most closely...,the usual reason their policies most closely a...,the usual reason their policy most closely ali...,"[▁The, ▁us, u, al, ▁reason, ▁-, ▁their, ▁polic..."
1996,1996,fqo3dlb,"it's not deep though, it is a surface level ob...",it not deep though it is a surface level obser...,it not deep though it be a surface level obser...,"[▁it, ', s, ▁not, ▁deep, ▁though, ,, ▁it, ▁is,..."
1997,1997,fhhct09,"**Hello u/UltimateWebRedditor, unfortunately y...",hello unfortunately your submission do blame c...,hello unfortunately your submission do blame c...,"[▁**, H, el, lo, ▁, u, /, U, l, t, im, ate, W,..."
1998,1998,h4s5us0,&gt; New conventional nuclear plants... exorbi...,new conventional nuclear plants exorbitant cos...,new conventional nuclear plant exorbitant cost...,"[▁&, gt, ;, ▁New, ▁con, ve, n, tion, al, ▁nucl..."


Comparison between the tokenizers

In [35]:
import pandas as pd
from tokenizers import Tokenizer, models, pre_tokenizers, trainers, normalizers
import sentencepiece as spm
from transformers import BertTokenizer
import spacy

# Sample dataframe
df_comments

# BPE Tokenizer
tokenizer_bpe = Tokenizer(models.BPE())
tokenizer_bpe.pre_tokenizer = pre_tokenizers.Whitespace()
trainer_bpe = trainers.BpeTrainer(vocab_size=10000, min_frequency=2, special_tokens=["<unk>", "<s>", "</s>", "<pad>", "<mask>"])
texts = df_comments['body'].tolist()
tokenizer_bpe.train_from_iterator(texts, trainer_bpe)
vocab_bpe = tokenizer_bpe.get_vocab()
vocab_size_bpe = len(vocab_bpe)

# Unigram Tokenizer
with open('text_data.txt', 'w') as f:
    for text in df_comments['body']:
        f.write(text + '\n')

spm.SentencePieceTrainer.Train('--input=text_data.txt --model_prefix=unigram --vocab_size=2000 --model_type=unigram')
sp_unigram = spm.SentencePieceProcessor()
sp_unigram.Load("unigram.model")
vocab_size_unigram = len(sp_unigram)

# WordPiece Tokenizer
tokenizer_wordpiece = BertTokenizer.from_pretrained('bert-base-uncased')
vocab_wordpiece = tokenizer_wordpiece.get_vocab()
vocab_size_wordpiece = len(vocab_wordpiece)

# spaCy Tokenizer
nlp_spacy = spacy.blank("en")
vocab_size_spacy = len(nlp_spacy.vocab)

# Collect the results
results = {
    "Tokenizer": ["BPE", "Unigram", "WordPiece", "spaCy"],
    "Vocab Size": [vocab_size_bpe, vocab_size_unigram, vocab_size_wordpiece, vocab_size_spacy]
}

# Create a DataFrame to display the results
df_results = pd.DataFrame(results)
print(df_results)

sentencepiece_trainer.cc(178) LOG(INFO) Running command: --input=text_data.txt --model_prefix=unigram --vocab_size=2000 --model_type=unigram
sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: text_data.txt
  input_format: 
  model_prefix: unigram
  model_type: UNIGRAM
  vocab_size: 2000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pa

   Tokenizer  Vocab Size
0        BPE       10000
1    Unigram        2000
2  WordPiece       30522
3      spaCy         764


/Users/fabianmahner/dlss-project24/venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
